# Import Libraries

In [35]:
%pip install --upgrade scikit-learn
%pip install python-dotenv
%pip install influxdb-client
%pip install pandas
%pip install pyyaml
%pip install numpy
%pip install matplotlib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


...

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [36]:
import io
import os
import time
import yaml
import warnings
import pandas as pd
import json
import hashlib
import matplotlib.pyplot as plt
import csv
from influxdb_client import InfluxDBClient
from influxdb_client.client.warnings import MissingPivotFunction
from dotenv import load_dotenv

import numpy as np
from functools import partial

warnings.simplefilter("ignore", MissingPivotFunction)

load_dotenv()

False

# Import Dataset

In [37]:
df_digi = pd.read_csv('dataset.csv')
df_dosen = pd.read_csv('dosen.csv')
df_sc = pd.read_csv('student_corner.csv')
df_eltron = pd.read_csv('half eltron.csv')

In [38]:
# Cleaning from invalid data
df_sc = df_sc[~df_sc["reference_point"].str.contains("RP31", na=False)]


In [39]:
df = pd.concat([df_digi, df_dosen, df_sc, df_eltron], ignore_index=True)

In [40]:
df.head()

,Unnamed: 0,bssid,channel,host,iteration,reference_point,rssi,ssid,time,topic,xr,yr
0,NaN,70:A7:41:DC:8E:55,1,LAUREN,1,RP29,-92,DTE Staff,2024-10-17T10:19:36.0867022Z,esp/topic,989,1037
1,NaN,70:A7:41:DC:8E:55,1,LAUREN,10,RP29,-94,DTE Staff,2024-10-17T10:23:26.6782315Z,esp/topic,989,1037
2,NaN,70:A7:41:DC:8E:55,1,LAUREN,11,RP29,-94,DTE Staff,2024-10-17T10:23:51.8041404Z,esp/topic,989,1037
3,NaN,70:A7:41:DC:8E:55,1,LAUREN,12,RP29,-90,DTE Staff,2024-10-17T10:24:13.6954786Z,esp/topic,989,1037
4,NaN,70:A7:41:DC:8E:55,1,LAUREN,13,RP29,-91,DTE Staff,2024-10-17T10:24:38.6661326Z,esp/topic,989,1037


In [41]:
df.columns

Index(['Unnamed: 0', 'bssid', 'channel', 'host', 'iteration',
       'reference_point', 'rssi', 'ssid', 'time', 'topic', 'xr', 'yr'],
      dtype='object')

## Drop Unnecessary Columns

In [42]:
df = df.drop(columns=['Unnamed: 0','host', 'topic'])

In [43]:
df.sample(10)

,bssid,channel,iteration,reference_point,rssi,ssid,time,xr,yr
2263,70:A7:41:DC:77:CD,1,19,RP66,-74,DTE Staff,2025-03-01T02:27:15.5186908Z,1826,1948
274,70:A7:41:DC:77:CD,11,6,RP29,-45,DTE Staff,2024-10-17T10:21:38.7158651Z,989,1037
63,72:A7:41:9C:8E:55,1,14,RP30,-91,DTE IoT,2024-10-17T10:34:37.1101766Z,989,1202
1646,72:A7:41:9C:8E:55,1,5,RP39,-73,DTE IoT,2024-10-23T07:45:17.2173303Z,2224,1037
1205,70:A7:41:DC:77:CD,11,17,RP32,-44,DTE Staff,2024-10-23T09:22:18.6175566Z,1219,1202
3283,72:A7:41:9C:77:CD,1,20,RP82,-68,DTE Student,2025-03-01T02:16:36.0037611Z,2000,838
2964,72:A7:41:9C:77:CD,1,1,RP90,-62,DTE Student,2025-03-01T05:00:52.6652095Z,2454,1393
1031,72:A7:41:9C:77:CD,11,19,RP19,-71,DTE Student,2024-10-23T04:16:02.5510164Z,989,1948
1904,70:A7:41:DC:77:CD,11,1,RP42,-58,DTE Staff,2024-10-23T07:09:45.0457192Z,2454,1202
481,72:A7:41:9C:77:CD,11,13,RP5,-67,DTE Student,2024-10-17T09:30:38.7287048Z,300,2130


In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5172 entries, 0 to 5171
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   bssid            5172 non-null   object
 1   channel          5172 non-null   int64 
 2   iteration        5172 non-null   int64 
 3   reference_point  5172 non-null   object
 4   rssi             5172 non-null   int64 
 5   ssid             5172 non-null   object
 6   time             5172 non-null   object
 7   xr               5172 non-null   int64 
 8   yr               5172 non-null   int64 
dtypes: int64(5), object(4)
memory usage: 363.8+ KB


In [45]:
df['bssid'].unique()

array(['70:A7:41:DC:8E:55', '72:A7:41:9C:8E:55', '70:A7:41:DC:77:CD',
       '72:A7:41:9C:77:CD'], dtype=object)

In [46]:
df['ssid'] = df['ssid'].replace('DTE IoT', 'DTE Staff')

In [47]:
df['ssid'].unique()

array(['DTE Staff', 'DTE Student'], dtype=object)

In [48]:
df['reference_point'].unique()

array(['RP29', 'RP30', 'RP37', 'RP4', 'RP5', 'RP6', 'RP1', 'RP10', 'RP2',
       'RP3', 'RP7', 'RP8', 'RP9', 'RP11', 'RP12', 'RP13', 'RP14', 'RP15',
       'RP16', 'RP17', 'RP18', 'RP19', 'RP20', 'RP31', 'RP32', 'RP33',
       'RP34', 'RP35', 'RP36', 'RP38', 'RP39', 'RP40', 'RP41', 'RP42',
       'RP49', 'RP53', 'RP54', 'RP55', 'RP56', 'RP58', 'RP59', 'RP60',
       'RP61', 'RP63', 'RP64', 'RP66', 'RP67', 'RP68', 'RP69', 'RP70',
       'RP71', 'RP72', 'RP74', 'RP75', 'RP76', 'RP77', 'RP80', 'RP81',
       'RP82', 'RP85', 'RP88', 'RP89', 'RP90', 'RP113', 'RP114', 'RP115',
       'RP118', 'RP119', 'RP120', 'RP100', 'RP103', 'RP104', 'RP105',
       'RP108', 'RP109', 'RP28', 'RP93', 'RP94', 'RP95', 'RP98', 'RP99',
       'RP110'], dtype=object)

In [49]:
import pandas as pd

df['bssid'] = df['bssid'].str.lower()

# BSSID mapping for classification
bssid_classification = {
    '70:a7:41:dc:8e:55': 'ap_dosen',     # Dosen (Staff)
    '72:a7:41:9c:8e:55': 'ap_dosen',     # Dosen (Student)
    '70:a7:41:dc:77:cd': 'ap_digilab',   # Digilab (Staff)
    '72:a7:41:9c:77:cd': 'ap_digilab'    # Digilab (Student)
}

# Map BSSID to AP classification
df['AP_class'] = df['bssid'].map(bssid_classification)

# Check if there are any NaN values (due to missing BSSID mapping)
missing_bssid = df[df['AP_class'].isna()]
if not missing_bssid.empty:
    print("These BSSIDs were not classified:")
    print(missing_bssid)

# Pivot the dataframe
pivot_df = df.pivot_table(index=['time', 'reference_point', 'iteration', 'ssid', 'bssid', 'channel', 'xr', 'yr'], columns='AP_class', values='rssi').reset_index()

# Rename columns for clarity
pivot_df.columns = ['time', 'reference_point', 'iteration', 'ssid', 'bssid', 'channel', 'xr', 'yr'] + ['rssi_' + col for col in pivot_df.columns[8:]]

# Show the result
print(pivot_df)

                              time reference_point  iteration         ssid  \
0     2024-10-17T08:38:31.9525624Z             RP1          3  DTE Student   
1     2024-10-17T08:39:38.6086206Z             RP1          1    DTE Staff   
2     2024-10-17T08:39:45.8326223Z             RP1          1  DTE Student   
3     2024-10-17T08:40:03.1574484Z             RP1          2  DTE Student   
4     2024-10-17T08:40:08.1701809Z             RP1          2    DTE Staff   
...                            ...             ...        ...          ...   
5167  2025-03-03T08:57:04.2900809Z            RP94          4    DTE Staff   
5168  2025-03-03T08:57:20.4820244Z            RP94          5  DTE Student   
5169   2025-03-03T08:57:23.550665Z            RP94          5    DTE Staff   
5170  2025-03-03T08:57:26.5138023Z            RP94          5    DTE Staff   
5171  2025-03-03T08:57:29.4971955Z            RP94          5    DTE Staff   

                  bssid  channel   xr    yr  rssi_ap_digilab  r

In [50]:
pivot_df.sample(10)

,time,reference_point,iteration,ssid,bssid,channel,xr,yr,rssi_ap_digilab,rssi_ap_dosen
3543,2025-03-01T03:13:09.5957577Z,RP56,9,DTE Student,72:a7:41:9c:77:cd,1,1479,1948,-65.0,NaN
1400,2024-10-23T07:33:49.1991921Z,RP40,3,DTE Staff,70:a7:41:dc:8e:55,1,2224,1202,NaN,-89.0
1851,2024-10-23T09:18:08.2232835Z,RP32,7,DTE Staff,72:a7:41:9c:8e:55,1,1219,1202,NaN,-94.0
4814,2025-03-03T07:59:42.1229083Z,RP108,20,DTE Staff,70:a7:41:dc:8e:55,11,1120,469,NaN,-88.0
417,2024-10-17T10:14:02.7286326Z,RP10,14,DTE Student,72:a7:41:9c:77:cd,11,529,2130,-66.0,NaN
5008,2025-03-03T08:26:45.0526471Z,RP99,17,DTE Staff,72:a7:41:9c:8e:55,11,720,639,NaN,-98.0
3250,2025-03-01T02:23:06.9928588Z,RP66,8,DTE Staff,72:a7:41:9c:8e:55,11,1826,1948,NaN,-95.0
3072,2025-03-01T01:59:24.2599407Z,RP81,2,DTE Staff,72:a7:41:9c:8e:55,11,2000,639,NaN,-76.0
4785,2025-03-03T07:55:52.1113845Z,RP108,10,DTE Student,72:a7:41:9c:77:cd,6,1120,469,-51.0,NaN
1356,2024-10-23T07:27:56.941797Z,RP41,12,DTE Staff,70:a7:41:dc:8e:55,1,2454,1037,NaN,-64.0


In [51]:
# Save the final dataframe
pivot_df.to_csv('ui_data_2.csv', index=False)
print(f"Data saved to {'ui_data.csv'}")

# Print some statistics
print(f"Date range: {pivot_df['time'].min()} to {pivot_df['time'].max()}")
print(f"Total records: {len(pivot_df)}")
print(f"X-coordinate range: {pivot_df['xr'].min()} to {pivot_df['xr'].max()}")
print(f"Y-coordinate range: {pivot_df['yr'].min()} to {pivot_df['yr'].max()}")

Data saved to ui_data.csv
Date range: 2024-10-17T08:38:31.9525624Z to 2025-03-03T08:57:29.4971955Z
Total records: 5172
X-coordinate range: 300 to 2454
Y-coordinate range: 299 to 2130
